In [6]:
import cv2
import mediapipe as mp
import numpy as np
import os

In [7]:
from win32api import GetSystemMetrics
w =  GetSystemMetrics(0)
h = GetSystemMetrics(1)
print("Width =", GetSystemMetrics(0))
print("Height =", GetSystemMetrics(1))

Width = 1280
Height = 720


In [8]:

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_holistic = mp.solutions.holistic
mp_pose = mp.solutions.pose

mpPose = mp.solutions.pose
pose = mpPose.Pose()
mpDraw = mp.solutions.drawing_utils

numPoses = 0

cap = cv2.VideoCapture(0)

while True: 

    ret,frame = cap.read()

    with mp_pose.Pose(static_image_mode=True, 
            model_complexity=2,
                    enable_segmentation=True) as pose:
            

        #image = cv2.imread(frame)
        #display(image)


        results = pose.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        

    segmented_image = frame.copy()

    tightness = 0.3


    if np.stack((results.segmentation_mask,) * 3, axis=-1).any() == None: 
        continue

    condition = np.stack((results.segmentation_mask,) * 3, axis=-1) > tightness

    bg_image = np.zeros(frame.shape, dtype=np.uint8)
    figure = np.zeros(frame.shape, dtype=np.uint8)

    bg_image[:] = [255, 255, 255]
    figure[:] = [0, 0, 0]


    segmented_image = np.where(condition, figure, bg_image)
    segmented_image = cv2.resize(segmented_image, (w,h))
    
    cv2.imshow("pose", segmented_image)
    path = "C:\Computer\Programming\opencv\c\poses"
    if cv2.waitKey(1) & 0xFF == ord('q'):
            cv2.destroyAllWindows()
            cap = None
            break

    if cv2.waitKey(1) & 0xFF == ord('y'):

        if numPoses < 1: 
            numPoses += 1

            filename = f"pose{numPoses}.jpg"

            cv2.imwrite(os.path.join(path , filename), segmented_image)

        else: 
            cv2.destroyAllWindows()
            cap = None
            break